In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


from datetime import datetime, date
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
FILES_DIR = './files/'
product = pd.read_csv(FILES_DIR + 'sampled_products.csv')
# stop_words_df = open('./files/stop_words.txt', 'r', encoding='utf-8')

In [13]:
pproduct = product.dropna()

In [14]:
cat_product = product.copy()

cat_product['cat'] = cat_product['category1_name'] + " " + cat_product['category2_name']
cat_product['cat'] = cat_product['cat'] + " " + cat_product['category3_name']
cat_product

,item_no,item_name,image_name,price,category1_code,category1_name,category2_code,category2_name,category3_code,category3_name,brand_no,brand_name,cat
0,++/211gtgspgB1TaPWEUQw==,베스트 코트\n( Vest Coat )_lavender,6842e3f320ef4459fac54dcea70c8245,1150000.0,243100100,의류,243101100,아우터,243101103,코트,4308.0,FOCEPS,의류 아우터 코트
1,++4eWFkX4oCtVYiKi34/Dw==,21SS 넥 페이스 패치 기모 맨투맨 블랙 CI0062 AHJ,1bdc06804580fddfbe46d4e2af140f08,466830.0,243100100,의류,243102100,상의,243102103,티셔츠,9302.0,Acne Studios_바이스트,의류 상의 티셔츠
2,++BoE+rYvLTJEAXMQFQLpQ==,[서울웰컴티 선물 세트] 일상 옥수수차,cb4a9bb103f9bd84d5d126ad4acec74b,15000.0,249100100,식품,249105100,음료,249105105,차류,4775.0,Urbanbookshop,식품 음료 차류
3,++RIzj1vpFLWgoX7LgCfLQ==,PLAIN SQUARE BAG S (IV),4c3f75e886c8114d7d00d19a4f0c34d5,118000.0,244100100,잡화,244102100,가방,244102104,토트백,2546.0,zisoo,잡화 가방 토트백
4,++W7/YMlPMwbAfyyJZEuIw==,클리어 바디미스트 150ml,5f136d4e25c837c3b658de16ac2156d9,27000.0,247100100,화장품/미용,247109100,바디케어,247109104,바디미스트,12986.0,밀크바오밥,화장품/미용 바디케어 바디미스트
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283321,zznjWkK64Fsflp1BbqeXQA==,SUMMER WOOL DOLMAN SLEEVE KNIT CARDIGAN,6e4bac52c12f2b2b923404c926fe404f,145000.0,243100100,의류,243102100,상의,243102102,카디건,14321.0,Carriere,의류 상의 카디건
283322,zzq8OW9uW0wofJhaKzp/Fg==,Silket Cotton Jersey T-Shirt Grey,569af51a0a61f7156b34b5cf251b82f0,98000.0,243100100,의류,243102100,상의,243102103,티셔츠,2085.0,aheit,의류 상의 티셔츠
283323,zzxE9xkXnKM0B4Y+hev34w==,[본사직영] 골라클래식 남성 COASTER HIGH (WHITE) GC_MA205_WW,3ca49f5d240325423bdc90942e2d61c5,75000.0,244100100,잡화,244101100,신발,244101115,스니커즈,4440.0,GOLA,잡화 신발 스니커즈
283324,zzzTNW8escss3zrnpuGMhQ==,21SS 마몬트 퀄팅 숄더/크로스백 448065 DTD1T 1000,00acf5906a360209ea248ae8933a2408,1743300.0,244100100,잡화,244102100,가방,244102103,크로스백,9410.0,GUCCI_바이스트,잡화 가방 크로스백


In [15]:
cat_product = cat_product.sample(frac=0.05, random_state=42)
cat_product = cat_product.drop_duplicates()
cat_product = cat_product.dropna()
cat_product['cat'] = cat_product['cat'].fillna('')

tfidf = TfidfVectorizer() 
tfidf_matrix = tfidf.fit_transform(cat_product['cat'])

In [16]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = pd.DataFrame(cosine_sim, index=cat_product.index, columns=cat_product.index)

In [17]:
indices = pd.Series(cat_product.index, index=cat_product['item_name'])
indices

item_name
Stream Dress_Sky                                                195722
UNISEX CUT-OFF DT LOGO SWEATSHORTS LIGHT MGREY_M_UDPA1E111G1      8322
당일발송)와이쓰리 GT5256 유틸리티 로고 스윔팬츠                                     3745
라탄 타원 미니 바구니 / 오발 트레이 정리함 인테리어 소품                               122497
버켄스탁 키즈 아리조나 일렉트릭 메탈릭실버 1019147                                 109624
                                                                 ...  
내열유리 머그잔1P 430ml (M-01)                                         184816
Modern Square Chain Bracelet                                    276474
Reversible linen blouse - black                                  49722
ABSL AIR-90 전용 카본필터                                              60902
플루크 마운틴 그래픽 실루엣 반팔티셔츠 FST151 / 4color M                          50215
Length: 14165, dtype: int64

In [18]:
def content_recommender(item_name, n_of_recomm):
  idx = indices[item_name]
  sim_scores = cosine_sim[idx].sort_values(ascending=False)[1:n_of_recomm]

  return cat_product.loc[sim_scores.index]['item_name']

print(content_recommender('Stream Dress_Sky', 5))
print('-' * 30)
print(content_recommender('Modern Square Chain Bracelet', 10))

201214    Ann Shirring Check Dress in Khaki
254995           V-NECK FLARE ONEPIECE_NAVY
386                         하와이안 레트로 원피스_핑크
92369               floral color P dress PK
Name: item_name, dtype: object
------------------------------
221835                        Basic Bangle_Beige [YA103_BG]
205319    [Silver925] TN051 Sleek curved line simple bra...
226605                                             실버 체인 팔찌
25231             Half And Blue Cord Bracelet Ib61 [Silver]
56473                                            핑퐁 블랙 원석팔찌
76433     Smile blue green string Bracelet 실버코인 블루 그린 실팔...
20081         [판도라 PANDORA] 587125CZ 모먼트 슬라이딩 스네이크 체인 로즈 팔찌
208145                             Curvilinear Archives 39A
172348                                          트로피컬 더블링 팔찌
Name: item_name, dtype: object
